In [4]:
# Load pandas module
import pandas as pd
import duckdb

### Step 1:
Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.

In [12]:
# Connect to database
conn = duckdb.connect("database/database.db")
conn

In [13]:
# Create cursor
cur = conn.cursor()

In [23]:
# Check our tables are available
cur.execute("SHOW TABLES").df()

,name
0,Bird_eggs
1,Bird_nests
2,Camp_assignment
3,Crop_calendar
4,Crop_calendar_2
5,Crop_yield
6,Crop_yield_2
7,Personnel
8,Site
9,Site_avg_snowcover


In [22]:
# Let's first view the variables in the Bird_eggs table
cur.execute("PRAGMA table_info('Bird_eggs');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,Book_page,VARCHAR,False,None,False
1,1,Year,INTEGER,True,None,False
2,2,Site,VARCHAR,True,None,False
3,3,Nest_ID,VARCHAR,True,None,True
4,4,Egg_num,INTEGER,True,None,True
5,5,Length,FLOAT,True,None,False
6,6,Width,FLOAT,True,None,False


In [21]:
# View the species column
cur.execute("PRAGMA table_info('Species');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,Code,VARCHAR,True,None,True
1,1,Common_name,VARCHAR,True,None,False
2,2,Scientific_name,VARCHAR,False,None,False
3,3,Relevance,VARCHAR,False,None,False


In [20]:
# View Bird eggs data
cur.execute("SELECT * FROM Bird_eggs LIMIT 5;").fetchdf()

,Book_page,Year,Site,Nest_ID,Egg_num,Length,Width
0,b14.6,2014,eaba,14eabaage01,1,39.139999,33.000000
1,b14.6,2014,eaba,14eabaage01,2,41.509998,33.389999
2,b14.6,2014,eaba,14eabaage01,3,48.290001,33.400002
3,b14.6,2014,eaba,14eabaagl01,1,47.560001,32.360001
4,b14.6,2014,eaba,14eabaagl01,2,48.130001,32.400002
